In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [1]:
%%shell
conda create --name env python=3.8 -y
source activate env
conda install pytorch -c pytorch-lts -c nvidia

python -m pip install -U pip
pip install --upgrade setuptools
pip install transformers==4.18.0

pip install datasets accelerate scipy scikit-learn

Solving environment: | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    setuptools-65.6.3          |     pyhd8ed1ab_0         619 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         761 KB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge None
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu None
  bzip2              conda-forge/linu

In [2]:
%%shell
source activate env
git clone https://github.com/schwartz-lab-NLP/papa.git

Cloning into 'papa'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 120 (delta 40), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (120/120), 143.52 KiB | 3.88 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [3]:
!ls papa/transformers/src/transformers

modeling_utils.py  models  papa_modules.py


In [4]:
# Copy local transformers content to installed transformers location
!cp -R papa/transformers/src/transformers/* /usr/local/envs/env/lib/python3.8/site-packages/transformers/

In [5]:
!ls /usr/local/envs/env/lib/python3.8/site-packages/transformers/ | grep papa_modules.py

papa_modules.py


In [ ]:
# !pip install huggingface_hub
# !huggingface-cli login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hu

In [6]:
%%shell
source activate env
pip install sentencepiece   # For deberta tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00


## MODEL NAMES :

`DeBERTa` : microsoft/deberta-base , microsoft/deberta-large

`BERT` : bert-base-uncased , bert-large-uncased

`RoBERTa` : roberta-base , roberta-large

`numpy`>=1.20.3 but <= 1.24.0 would be needed for the `DeBERTa` code to run.

In [7]:
%%shell
source activate env
pip uninstall numpy -y
pip install numpy==1.20.3

Found existing installation: numpy 1.24.1
Uninstalling numpy-1.24.1:
  Successfully uninstalled numpy-1.24.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 74.1 MB/s eta 0:00:00


In [16]:
%%shell
source activate env
cd papa/transformers/papa_scripts

MODEL=microsoft/deberta-base
TASK=sst2

python3 run_papa_glue_avgs_creator.py --model_name_or_path ${MODEL}  --task_name ${TASK}  --max_length 64    --per_device_train_batch_size 8   --output_dir ../../constant-matrices/ --cache_dir ../../cache/ --use_papa_preprocess true  --pad_to_max_length

01/11/2023 14:11:39 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no

Dataset glue downloaded and prepared to /content/papa/transformers/papa_scripts/../../cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.
100% 3/3 [00:00<00:00, 574.20it/s]
loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-base",
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "sst2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "ma

In [ ]:
%%shell
source activate env
cd papa/transformers/papa_scripts

MODEL=microsoft/deberta-base
TASK=sst2

python3 run_papa_glue.py --model_name_or_path ${MODEL} --task_name ${TASK} --do_eval --max_seq_length 64 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --output_dir ../../sorted-heads/ --cache_dir ../../cache/  --do_train --num_train_epochs 15.0 --learning_rate 2e-5 --lr_scheduler_type constant --disable_tqdm true --evaluation_strategy epoch --save_strategy no --use_papa_preprocess --use_freeze_extract_pooler true --static_heads_dir ../../constant-matrices/  --save_total_limit 0 --sort_calculating True

01/11/2023 14:18:02 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/11/2023 14:18:02 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=True,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=

### Run the following cell for base models (total heads = 12 (heads_per_layer) * 12 (num_layers) = 144)

In [ ]:
%%shell

mkdir -p papa/results/
mkdir -p papa/results/0
mkdir -p papa/results/72
mkdir -p papa/results/126
mkdir -p papa/results/135
mkdir -p papa/results/144

In [ ]:
%%shell
source activate env
cd papa/transformers/papa_scripts

MODEL=microsoft/deberta-base
TASK=sst2

for static_heads_num in 0 72 126 135 144
 do
    python3 run_papa_glue.py --model_name_or_path ${MODEL} --task_name ${TASK} --do_eval --max_seq_length 64 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --output_dir ../../output/ --overwrite_output_dir --cache_dir ../../cache/ --do_train --num_train_epochs 15.0 --learning_rate 2e-5 --lr_scheduler_type constant --disable_tqdm true --evaluation_strategy epoch --save_strategy no --use_papa_preprocess --grad_for_classifier_only true --use_freeze_extract_pooler true --static_heads_dir ../../constant-matrices/ --static_heads_num ${static_heads_num} --save_total_limit 0 --sorting_heads_dir ../../sorted-heads/
    cp ../../output/README.md ../../results/${static_heads_num}/
 done

### Run the following cell for large models (total heads = 16 (heads_per_layer) * 24 (num_layers) = 384)

In [ ]:
%%bash

mkdir -p papa/results/
mkdir -p papa/results/0
mkdir -p papa/results/192
mkdir -p papa/results/336
mkdir -p papa/results/360
mkdir -p papa/results/384

In [ ]:
%%bash
source activate env
cd papa/transformers/papa_scripts

MODEL=bert-large-uncased
TASK=cola

for static_heads_num in 0 192 336 360 384
 do
    python3 run_papa_glue.py --model_name_or_path ${MODEL} --task_name ${TASK} --do_eval --max_seq_length 64 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --output_dir ../../output/ --overwrite_output_dir --cache_dir ../../cache/ --do_train --num_train_epochs 15.0 --learning_rate 2e-5 --lr_scheduler_type constant --disable_tqdm true --evaluation_strategy epoch --save_strategy no --use_papa_preprocess --grad_for_classifier_only true --use_freeze_extract_pooler true --static_heads_dir ../../constant-matrices/ --static_heads_num ${static_heads_num} --save_total_limit 0 --sorting_heads_dir ../../sorted-heads/
    cp ../../output/README.md ../../results/${static_heads_num}/
 done

### Save the outputs 

In [ ]:
%%shell
cd papa

MODEL=deberta-base
TASK=sst2

zip -r /content/${MODEL}_${TASK}_constant-matrices.zip constant-matrices/
zip -r /content/${MODEL}_${TASK}_sorted-heads.zip sorted-heads/
zip -r /content/${MODEL}_${TASK}_results.zip results/
zip -r /content/${MODEL}_${TASK}_output.zip output/

In [ ]:
from google.colab import files

MODEL="deberta-base"
TASK="sst2"

files.download(f"/content/{MODEL}_{TASK}_constant-matrices.zip")
files.download(f"/content/{MODEL}_{TASK}_sorted-heads.zip")
files.download(f"/content/{MODEL}_{TASK}_results.zip")
files.download(f"/content/{MODEL}_{TASK}_output.zip")

In [15]:
# Clear directory before running each new model
%rm -rf papa/cache/*
%rm -rf papa/constant-matrices/*
%rm -rf papa/sorted-heads/*
%rm -rf papa/output/*
%rm -rf papa/results/*

## Analysis

In [ ]:
%%bash
source activate env
python

import torch
attention_sums = torch.load('papa/transformers/papa_scripts/outputs/attention_sums.pt')
print(f"Shape of attention_sums : {attention_sums.size()}")         # (num_layers=12, num_heads=12, )

print(attention_sums[0][0].min())
# DIMENSIONS :

# num_layers = 12
# num_heads = 12
# num_input_tokens (n) = 64

Shape of attention_sums : torch.Size([12, 12, 64, 64])
tensor(0.)
